# Real Estate Property Lookup Agent Tool

This notebook demonstrates how to build a LangChain agent tool for real estate property lookup. The tool accepts locale, price range, and number of rooms as input, and returns a JSON list of up to 10 property listings with key attributes suitable for display or use by other agents.

In [ ]:
# Import Required Libraries
from typing import List, Dict, Any
import random
import json

# If using LangChain, import relevant modules (mocked here for demonstration)
try:
    from langchain.tools import Tool
    from langchain.agents import initialize_agent, AgentType
    from langchain.llms import OpenAI
except ImportError:
    Tool = None
    initialize_agent = None
    AgentType = None
    OpenAI = None

## Define Property Lookup Tool Function

We will implement a function that takes locale, price range, and number of rooms as input, queries a mock property data source, and returns up to 10 property listings as a JSON list with key attributes (address, price, rooms, description, image_url).

In [ ]:
def property_lookup(locale: str, price_min: int, price_max: int, num_rooms: int) -> List[Dict[str, Any]]:
    """
    Mock function to return up to 10 property listings based on input criteria.
    """
    # Mock property data
    mock_properties = []
    for i in range(30):
        price = random.randint(50000, 1000000)
        rooms = random.randint(1, 6)
        property_locale = random.choice([locale, "OtherTown", "NearbyCity"])
        mock_properties.append({
            "address": f"{i+1} Main St, {property_locale}",
            "price": price,
            "rooms": rooms,
            "description": f"A lovely {rooms}-room property in {property_locale}.",
            "image_url": f"https://example.com/property_{i+1}.jpg"
        })
    # Filter by criteria
    filtered = [p for p in mock_properties if p["price"] >= price_min and p["price"] <= price_max and p["rooms"] == num_rooms and p["address"].endswith(locale)]
    # Return up to 10
    return filtered[:10]

# Example usage:
example_results = property_lookup("SampleTown", 200000, 800000, 3)
print(json.dumps(example_results, indent=2))

## Integrate Tool with LangChain Agent

Now, we will create a LangChain agent and register the property lookup tool so the agent can use it to answer queries.

In [ ]:
# Register the property lookup tool with LangChain agent

def property_lookup_tool_func(input_str: str) -> str:
    # Parse input string (simple parsing for demonstration)
    import re
    locale = re.search(r"locale=([^,]+)", input_str)
    price_min = re.search(r"price_min=(\d+)", input_str)
    price_max = re.search(r"price_max=(\d+)", input_str)
    num_rooms = re.search(r"num_rooms=(\d+)", input_str)
    locale = locale.group(1) if locale else "SampleTown"
    price_min = int(price_min.group(1)) if price_min else 0
    price_max = int(price_max.group(1)) if price_max else 1_000_000
    num_rooms = int(num_rooms.group(1)) if num_rooms else 3
    results = property_lookup(locale, price_min, price_max, num_rooms)
    return json.dumps(results)

# Mock LLM for demonstration if OpenAI is not available
class MockLLM:
    def __call__(self, prompt):
        return "locale=SampleTown, price_min=200000, price_max=800000, num_rooms=3"

llm = OpenAI() if OpenAI else MockLLM()

if Tool and initialize_agent and AgentType:
    property_tool = Tool(
        name="PropertyLookup",
        func=property_lookup_tool_func,
        description="Lookup real estate properties by locale, price range, and number of rooms."
    )
    agent = initialize_agent([property_tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
else:
    property_tool = None
    agent = None

## Test Agent with Sample Inputs

Let's test the agent with example queries specifying locale, price range, and number of rooms to verify correct tool invocation and output.

In [ ]:
# Test the agent with a sample query

sample_query = "Find properties in SampleTown with 3 rooms between $200,000 and $800,000."

if agent:
    response = agent.run(sample_query)
    print("Agent Output:", response)
else:
    # Directly call the tool function for demonstration
    input_str = "locale=SampleTown, price_min=200000, price_max=800000, num_rooms=3"
    response = property_lookup_tool_func(input_str)
    print("Tool Output:", response)

# Parse the JSON output for further display
parsed_results = json.loads(response)

## Format and Display Agent Output

Format the agent's JSON output for readability and display the property listings in a user-friendly way.

In [ ]:
# Display the property listings in a user-friendly way
import pandas as pd
from IPython.display import display, HTML

if parsed_results:
    df = pd.DataFrame(parsed_results)
    display(HTML(df.to_html(index=False)))
else:
    print("No properties found matching the criteria.")